In [1]:
from pathlib import Path

import cv2
import numpy as np
import rasterio
from tqdm import tqdm
from skimage import morphology

In [2]:
def run(input_raster, output_raster):
    src = rasterio.open(input_raster)
    src_img = src.read(1)
    profile = src.meta.copy()
    profile.update({'compress': 'deflate'})
    src.close()
    # 连通组件分析

    kernel = morphology.square(3)  # 3*3的正方形腐蚀核
    img_eroded = morphology.erosion(src_img, kernel)  # 先腐蚀
    img_dilated = morphology.dilation(img_eroded, kernel)  # 再膨胀
    edge_dilated = img_dilated - img_eroded  # 膨胀后的图像减去腐蚀后的图像，得到边缘
    edge_dilated = morphology.skeletonize(edge_dilated)
    xs_, ys_ = np.where(edge_dilated == 1)

    # 腐蚀后计算连通域
    _, labels_eroded, _, _ = cv2.connectedComponentsWithStats(img_eroded, connectivity=8)

    with tqdm(range(len(xs_)), desc='Progress') as tbar:
        # 对于每一个可能的关键点
        for x, y in zip(xs_, ys_):
            # 获取该点在腐蚀图像中的邻域
            neighborhood = set(labels_eroded[x - 1:x + 2, y - 2:y + 3].reshape(-1))
            neighborhood.update(labels_eroded[x - 2, y - 1:y + 2].reshape(-1))
            neighborhood.update(labels_eroded[x + 2, y - 1:y + 2].reshape(-1))

            # 如果邻域包含不同的连通域
            if len(neighborhood) > 2:  # 包括背景0
                # 那么这个点就可能是关键点
                img_dilated[x, y] = 0

            tbar.update()

    with rasterio.open(output_raster, 'w', **profile) as dst:
        dst.write(img_dilated, 1)


In [3]:
# 设置目录路径和文件后缀
dir_path = Path(r'D:\UAV_DATA_NEW\output\1_UTM47N')
output_path = Path(r'D:\UAV_DATA_NEW\output\2_dilated')

# 循环处理每个文件
for input_raster in dir_path.glob(f'*.tif'):
    file_name = input_raster.stem[0:6]
    output_raster = output_path / f'{file_name}_dilated.tif'
    print(output_raster.name)
    run(input_raster, output_raster)


061301_dilated.tif


Progress: 100%|██████████| 6890679/6890679 [00:24<00:00, 282532.29it/s]


061302_dilated.tif


Progress: 100%|██████████| 10887469/10887469 [00:42<00:00, 258051.87it/s]


061303_dilated.tif


Progress: 100%|██████████| 23307477/23307477 [01:21<00:00, 285623.05it/s]


061306_dilated.tif


Progress: 100%|██████████| 3098505/3098505 [00:10<00:00, 283061.75it/s]


061308_dilated.tif


Progress: 100%|██████████| 1983356/1983356 [00:06<00:00, 287962.54it/s]


In [4]:
# def run(input_raster, output_raster):
#     src = rasterio.open(input_raster)
#     src_img = src.read(1)
#     profile = src.profile
#     src.close()
#     # 连通组件分析
#     kernel = morphology.square(3)  # 3*3的正方形腐蚀核
#     img_eroded = morphology.erosion(src_img, kernel)  # 先腐蚀
#     img_dilated = morphology.dilation(img_eroded, kernel)  # 再膨胀
#     edge_dilated = img_dilated - img_eroded  # 膨胀后的图像减去腐蚀后的图像，得到边缘
#     skeleton = morphology.skeletonize(edge_dilated)  # 骨架化
#     sk_obj = Skeleton(skeleton)  # 骨架化对象

#     branch_data = summarize(sk_obj)

#     # j2e = branch_data.loc[branch_data['branch-type'] == 1] # j2e主要是被截掉的，通常不需要考虑
#     j2j = branch_data.loc[branch_data['branch-type'] == 2]  # 交合点到交合点的枝干 junction to junction
#     j2j = j2j[j2j['branch-distance'] <= 1.5]  # 寻找那些处于交点上的通路

#     all_points = [sk_obj.path_coordinates(index) for index in j2j.index]
#     intersected_points = np.unique(np.vstack(all_points), axis=0)
#     img_dilated[intersected_points[:, 0], intersected_points[:, 1]] = 0

#     with rasterio.open(output_raster, 'w', **profile) as dst:
#         dst.write(img_dilated, 1)

In [5]:
input_raster = r'D:\UAV_DATA_NEW\output\1_UTM47N\061301_UTM47N.tif'

src = rasterio.open(input_raster)
src_img = src.read(1)
profile = src.profile
src.close()
# 连通组件分析

kernel = morphology.square(3)  # 3*3的正方形腐蚀核
img_eroded = morphology.erosion(src_img, kernel)  # 先腐蚀
img_dilated = morphology.dilation(img_eroded, kernel)  # 再膨胀
edge_dilated = img_dilated - img_eroded  # 膨胀后的图像减去腐蚀后的图像，得到边缘
edge_dilated = morphology.skeletonize(edge_dilated)
xs_, ys_ = np.where(edge_dilated == 1)

# 腐蚀后计算连通域
_, labels_eroded, _, _ = cv2.connectedComponentsWithStats(img_eroded, connectivity=8)

with tqdm(range(len(xs_)), desc='Progress') as tbar:
    # 对于每一个可能的关键点
    for x, y in zip(xs_, ys_):
        # 获取该点在腐蚀图像中的方形邻域
        neighborhood = set(labels_eroded[x - 1:x + 2, y - 2:y + 3].reshape(-1))
        neighborhood.update(labels_eroded[x - 2, y - 1:y + 2].reshape(-1))
        neighborhood.update(labels_eroded[x + 2, y - 1:y + 2].reshape(-1))

        # 如果邻域包含不同的连通域
        if len(neighborhood) > 2:  # 包括背景0
            # 那么这个点就可能是关键点
            img_dilated[x, y] = 0
        tbar.update()


with rasterio.open(r'D:\UAV_DATA_NEW\output\061301_dilated_p.tif', 'w', **profile) as dst:
    dst.write(img_dilated, 1)

Progress: 100%|██████████| 6890679/6890679 [00:26<00:00, 263017.23it/s]
